## 加载工具包

In [15]:
import time
import re
import math
import numpy as np
from six.moves import urllib
import tensorflow as tf

## 读取数据

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
path = 'C:/Users/onlooker/Documents/deeplearning_projects/cats_dogs_classification/dataset'
training_set = train_datagen.flow_from_directory(path+'/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory(path+'/test_set',
                                            target_size = (64, 64),
                                            batch_size = 2000,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## 创建简单卷积网络

In [3]:
#创建权重变量
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.02)
  return tf.Variable(initial)
#创建偏差
def bias_variable(shape):
  initial = tf.constant(0.001, shape=shape)
  return tf.Variable(initial)

#卷积函数
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
#池化函数
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [4]:
graph = tf.Graph()
with graph.as_default():
    #变量定义
    #定义输入数据
    x = tf.placeholder(tf.float32, [None, 64, 64, 3])
    #期望输出标签
    y_ = tf.placeholder(tf.float32, [None,1])
    #丢弃因子
    keep_prob = tf.placeholder("float")

In [6]:
#两层的卷积网络
with graph.as_default():
    with tf.name_scope("conv1"):
        #第一层卷积
        W_conv1 = weight_variable([3, 3, 3, 32])
        b_conv1 = bias_variable([32])
        #卷积后采用Relu函数激活
    h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
    #进行池化
    h_pool1 = max_pool_2x2(h_conv1)
    with tf.name_scope("conv2"):
        #第二层卷积
        W_conv2 = weight_variable([3, 3, 32, 32])
        b_conv2 = bias_variable([32])
        #卷积后采用Relu函数激活
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        #进行池化
        h_pool2 = max_pool_2x2(h_conv2)

In [9]:
#全连接层
with graph.as_default():
    with tf.name_scope("fully_connected"):
        W_fc1 = weight_variable([16*16 * 32, 128])
        b_fc1 = bias_variable([128])
        h_pool2_flat = tf.reshape(h_pool2, [-1, 16*16*32])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    #丢弃，为了增强泛化能力
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [13]:
#输出层，softmax
#全连接层
with graph.as_default():
    with tf.name_scope("output"):
        W_fc2 = weight_variable([128, 1])
        b_fc2 = bias_variable([1])
    y_conv=tf.nn.sigmoid(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## 损失函数以及梯度下降算法

In [15]:
with graph.as_default():
    #损失函数
    cross_entropy = -tf.reduce_mean(y_*tf.log(y_conv) + (1-y_)*tf.log(1-y_conv))
    #梯度下降法，使用Adam算法
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
     #Decaying Learning Rate
    #cur_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
    #starter_learning_rate = 0.001
    #learning_rate = tf.train.exponential_decay(starter_learning_rate, cur_step, 100000, 0.96, staircase=True)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy, global_step=cur_step)
    #计算准确度
    preds = y_conv > 0.5
    correct_prediction = tf.equal(tf.cast(preds, tf.int32), tf.cast(y_, tf.int32))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [16]:
accuracy

<tf.Tensor 'Mean_1:0' shape=() dtype=float32>

In [17]:
num_steps = 501
epochs = 50
with tf.Session(graph=graph) as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for _ in range(epochs):
        for step in range(num_steps):
            #产生训练用样本集      
            batch_data, batch_labels = training_set.next()
            batch_labels = np.reshape(batch_labels, [-1,1])
            #数据传递给tensorflow
            feed_dict = {x : batch_data, y_ : batch_labels, keep_prob:0.5}
            sess.run(optimizer, feed_dict=feed_dict)
            if step%100 == 0:
                #每50次计算准确度
                result = sess.run(cross_entropy, feed_dict={x : batch_data, y_ : batch_labels, keep_prob:1})
                print('Cost:', result)   
    print('Testing Result.....')       
    batch_data, batch_labels = test_set.next()
    batch_labels = np.reshape(batch_labels, [-1,1])
    feed_dict = {x : batch_data, y_ : batch_labels, keep_prob:1}
    result = sess.run(accuracy, feed_dict)
    print('Accuracy:', result)

Cost: 0.693033
Cost: 0.693802
Cost: 0.688323
Cost: 0.683102
Cost: 0.678234
Cost: 0.695801
Cost: 0.610721
Cost: 0.661946
Cost: 0.604205
Cost: 0.617408
Cost: 0.65438
Cost: 0.551946
Cost: 0.621788
Cost: 0.572949
Cost: 0.732513
Cost: 0.618643
Cost: 0.692049
Cost: 0.610078
Cost: 0.606186
Cost: 0.59901
Cost: 0.495944
Cost: 0.676135
Cost: 0.678831
Cost: 0.574497
Cost: 0.58973
Cost: 0.419729
Cost: 0.563035
Cost: 0.665094
Cost: 0.571348
Cost: 0.547471
Cost: 0.521333
Cost: 0.487861
Cost: 0.51393
Cost: 0.619993
Cost: 0.601518
Cost: 0.619723
Cost: 0.563748
Cost: 0.481831
Cost: 0.406371
Cost: 0.464723
Cost: 0.528513
Cost: 0.478059
Cost: 0.8004
Cost: 0.550918
Cost: 0.614105
Cost: 0.499858
Cost: 0.6156
Cost: 0.49121
Cost: 0.491891
Cost: 0.648234
Cost: 0.578275
Cost: 0.526935
Cost: 0.581431
Cost: 0.500393
Cost: 0.466093
Cost: 0.496809
Cost: 0.526293
Cost: 0.578282
Cost: 0.49801
Cost: 0.505988
Cost: 0.545323
Cost: 0.507415
Cost: 0.613997
Cost: 0.535357
Cost: 0.661268
Cost: 0.627345
Cost: 0.386283
Cost:

KeyboardInterrupt: 

In [5]:
#Below code was referred to http://danijar.com/structuring-your-tensorflow-models/
import functools
TOWER_NAME = 'tower'
def define_scope(function):
    attribute = '_cache_' + function.__name__

    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
           # with tf.variable_scope(function.__name__):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)

    return decorator

In [42]:
#Define cifar model
class imageClassification:
    '''Define a basic model for image classification, the model
    Provides graph structure of tensorflow'''
    
    def __init__(self, image_holder, label_holder, batch_size=32, num_class=10):
        self.image_holder = image_holder
        self.label_holder = label_holder
        self.num_class = num_class
        self.weights = []
        self.batch_size = batch_size
        self.prediction
        self.optimize
        self.accuracy
        self.correct_num
        print('Initializing Image Classification Model!') 
    
    @define_scope
    def sigmoid(self):
        '''Calculate final softmax logits of 
        convolutional neural network'''
        logits = self.inference
        return tf.nn.sigmoid(logits)
        
    @define_scope
    def prediction(self):
        '''Calculate final softmax logits of 
        convolutional neural network'''
        sigmoid = self.sigmoid
        predictions = sigmoid > 0.5
        return tf.cast(predictions, tf.int32)
    
    @define_scope
    def optimize(self):
        '''Define cross entropy loss function and regularization'''
        #labels = tf.one_hot(self.label_holder, self.num_class, 1, 0)
        labels = tf.cast(self.label_holder, tf.float32)
        #Cross entropy
        #cross_entropy = -tf.reduce_mean(labels*
                                       #tf.log(self.prediction), name='cross_entropy')
        cross_entropy = -tf.reduce_mean(labels*tf.log(self.sigmoid) + (1-labels)*tf.log(1-self.sigmoid))
        
        #Regularization
        l2_loss = cross_entropy
        for i in range(len(self.weights)):
                l2_loss += tf.nn.l2_loss(self.weights[i])

        #Decaying Learning Rate
        cur_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
        starter_learning_rate = 0.4
        learning_rate = tf.train.exponential_decay(starter_learning_rate, cur_step, 100000, 0.96, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        #optimizer = tf.train.AdamOptimizer(1e-4)
        return optimizer.minimize(l2_loss, global_step=cur_step)
    
    @define_scope
    def accuracy(self):
        '''Calculate accuracy for each epoch of training'''
        #labels = tf.one_hot(self.label_holder, self.num_class, 1, 0)
        correct_prediction = tf.equal(self.label_holder, 
                                      self.prediction)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        return accuracy
    
    @define_scope
    def correct_num(self):
        '''Count correct predictions for testing part'''
        #labels = tf.one_hot(self.label_holder, self.num_class, 1, 0)
        correct_prediction = tf.equal(self.label_holder, 
                                      self.prediction)
        correct_ones = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))
        return correct_ones
    
    @define_scope
    def inference(self):
        """Build the clasification model.
        Args:
        images: Images returned from distorted_inputs() or inputs().
        Returns:
        Logits.
        """
        # We instantiate all variables using tf.get_variable() instead of
        # tf.Variable() in order to share variables across multiple GPU training runs.
        # If we only ran this model on a single GPU, we could simplify this function
        # by replacing all instances of tf.get_variable() with tf.Variable().
        #
        # conv1
        with tf.variable_scope('conv1') as scope:
            kernel = self._variable_with_weight_decay('weights',
                                                 shape=[3, 3, 3, 64],
                                                 wd=0.0)
            conv = tf.nn.conv2d(self.image_holder, kernel, [1, 1, 1, 1], padding='SAME')
            biases = self._variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv1 = tf.nn.relu(pre_activation, name=scope.name)
            self._activation_summary(conv1)
        # pool1
        pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                               padding='SAME', name='pool1')

        
        # conv2
        with tf.variable_scope('conv2') as scope:
            kernel = self._variable_with_weight_decay('weights',
                                                 shape=[3, 3, 64, 64],                                                 
                                                 wd=0.0)
            conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = self._variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv2 = tf.nn.relu(pre_activation, name=scope.name)
            self._activation_summary(conv2)      

        # pool2
        pool2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1],
                               strides=[1, 2, 2, 1], padding='SAME', name='pool2')
        
        # local3
        with tf.variable_scope('fully_connected1') as scope:
            # Move everything into depth so we can perform a single matrix multiply.
            reshape_tensor = tf.reshape(pool2, [self.batch_size, -1], name='reshape_tensor')
            dim = reshape_tensor.get_shape()[1].value
            weights = self._variable_with_weight_decay('weights', shape=[dim, 384],
                                                  wd=0.004)
            biases = self._variable_on_cpu('biases', [384], tf.constant_initializer(0.1))
            local3 = tf.nn.relu(tf.matmul(reshape_tensor, weights) + biases, name=scope.name)
            self._activation_summary(local3)
        # local4
        with tf.variable_scope('fully_connected2') as scope:
            weights = self._variable_with_weight_decay('weights', shape=[384, 192],
                                                  wd=0.004)
            biases = self._variable_on_cpu('biases', [192], tf.constant_initializer(0.1))
            local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
            self._activation_summary(local4)
        
        # linear layer(WX + b),
        # We don't apply softmax here because
        # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
        # and performs the softmax internally for efficiency.
        with tf.variable_scope('softmax_linear') as scope:
            weights = self._variable_with_weight_decay('weights', [192, self.num_class],
                                                  wd=0.0)
            biases = self._variable_on_cpu('biases', [self.num_class],
                                      tf.constant_initializer(0.0))
            softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
            self._activation_summary(softmax_linear)
        
        return softmax_linear
    
    def _activation_summary(self, x):
        """Helper to create summaries for activations.
        Creates a summary that provides a histogram of activations.
        Creates a summary that measures the sparsity of activations.
        Args:
        x: Tensor
        Returns:
        nothing
        """
        # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
        # session. This helps the clarity of presentation on tensorboard.
        tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)
        tf.summary.histogram(tensor_name + '/activations', x)
        tf.summary.scalar(tensor_name + '/sparsity',
                          tf.nn.zero_fraction(x))
        
    def _variable_on_cpu(self, name, shape, initializer):
        """Helper to create a Variable stored on CPU memory.
        Args:
        name: name of the variable
        shape: list of ints
        initializer: initializer for Variable
        Returns:
        Variable Tensor
        """
        with tf.device('/cpu:0'):
            with tf.variable_scope('weights'):
                dtype = tf.float32
                var = tf.get_variable(initializer=initializer(shape), dtype=dtype, name=name)
            #var = tf.get_variable(name, shape, initializer=initializer, dtype=dtype)
        return var

    def _variable_with_weight_decay(self, name, shape, wd):
        """Helper to create an initialized Variable with weight decay.
        Note that the Variable is initialized with a truncated normal distribution.
        A weight decay is added only if one is specified.
        Args:
        name: name of the variable
        shape: list of ints
        stddev: standard deviation of a truncated Gaussian
        wd: add L2Loss weight decay multiplied by this float. If None, weight
        decay is not added for this Variable.
        Returns:
        Variable Tensor
        """
        dtype = tf.float32
        var = self._variable_on_cpu(name,
                               shape,
                               tf.contrib.layers.xavier_initializer())
        if wd is not None:
            weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
            #tf.add_to_collection('losses', weight_decay)
            self.weights.append(weight_decay)
        return var

In [43]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    batch_size = 128
    num_steps = 8001
    image_holder = tf.placeholder(tf.float32, [batch_size, 64, 64, 3])
    label_holder = tf.placeholder(tf.int32, [batch_size])
    model = imageClassification(image_holder, label_holder, batch_size, num_class =1)

Initializing Image Classification Model!
